In [19]:
import os
import pandas as pd
import pygeos
import geopandas as gpd
from tqdm import tqdm

In [13]:
# SET PATHS
province = 'zeeland'
building_path = os.path.join("C:\\Data",'OSM','feather')
cbs_path = os.path.join("C://","Data","Bodemgebruik")

building_province_path = os.path.join(building_path,'{}.ft'.format(province))

In [14]:
df_buildings = pd.read_feather(building_province_path)
df_buildings.geometry = pygeos.from_wkb(df_buildings.geometry.values)

In [ ]:
%%time
BBG2015 = gpd.read_file(os.path.join(cbs_path,'BBG2015.shp'))
BBG2015 = BBG2015.to_crs("EPSG:4326")

In [16]:
df_BBG = pd.DataFrame(BBG2015[['Omschrijvi','geometry']].copy())
df_BBG.geometry = pygeos.from_shapely(df_BBG.geometry)

In [22]:
#overlay data
tqdm.pandas()
tree = pygeos.STRtree(df_BBG.geometry)

In [25]:
test = pygeos.centroid(df_buildings.geometry)[:100]

In [27]:
x = test.iloc[0]

In [28]:
tree.query(x,predicate='intersects')

array([], dtype=int64)

In [ ]:
df_buildings['BBG'] = pygeos.centroid(df_buildings.geometry).progress_apply(lambda x: df_BBG.iloc[tree.query(x,predicate='intersects')[0]]['Omschrijvi'])
